In [ ]:
# importing the necessary libraries
import pandas as pd 
import sqlalchemy as sml
import plotly.express as px

In [ ]:
df_prod = pd.read_csv('path to products.csv')
df_cust = pd.read_csv('path to customers.csv')
df_ord = pd.read_csv('path to orders.csv')

In [ ]:
# checking the number of duplicates in each column
all_cols_prod = df_prod.columns
all_cols_cust = df_cust.columns
all_cols_ord = df_ord.columns

for cols in all_cols_prod:
    print(f"Number of duplicates in {cols} column in products: {df_prod[cols].duplicated().sum()}")
print('-'*50)

for cols in all_cols_cust:
    print(f"Number of duplicates in {cols} column in customers: {df_cust[cols].duplicated().sum()}")
print('-'*50)

for cols in all_cols_ord:
    print(f"Number of duplicates in {cols} column in orders: {df_ord[cols].duplicated().sum()}")
print('-'*50)


In [ ]:
# establishing the connection to mysql server
engine = sml.create_engine("your_mysql_userand_local_host")
conn = engine.connect()
print("Connection established")

In [ ]:
# Dropping the duplicates in each dataframe
# on the basis of the primary key
df_prod.drop_duplicates(subset='product_code', inplace=True)
df_cust.drop_duplicates(subset='client_id', inplace=True)

In [ ]:
# writing the DataFrame to MySQL database
df_cust.to_sql('customers', con=conn, if_exists='append', index=False)
df_prod.to_sql('products', con=conn, if_exists='append', index=False)

# Rename the DataFrame columns before inserting
df_ord_renamed = df_ord.rename(columns={
    'order_number': 'order_id',
    'order_line_number': 'order_linenumber', 
    'status': 'order_status',
    'quantity': 'order_quantity',
    'price_each': 'price_sold'
})

# Then insert the renamed DataFrame
df_ord_renamed.to_sql('orders', con=conn, if_exists='append', index=False)

Sales analysis over time 

In [ ]:
# Monthly order trends 
monthly_trends = df_ord_renamed.groupby(df_ord_renamed['order_date'].str[:7]).agg(
    total_orders=('order_id', 'count'),
    total_quantity=('order_quantity', 'sum'),
    total_sales=('price_sold', 'sum')
).reset_index()

# plotting a line chart for monthly trends
import plotly.express as px
fig = px.line(
    monthly_trends,
    x='order_date',
    y=['total_orders', 'total_quantity', 'total_sales'],
    title='Monthly Order Trends',
    labels={'order_date.str[:7]': 'Month', 'value': 'Count/Sales'},
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Count/Sales/Orders",
    template='plotly_dark'
)

fig.show()
fig.write_html("Monthly order sales and quantity trends.html")

In [ ]:
# Yearly order trends
yearly_trends = df_ord_renamed.groupby(df_ord_renamed['order_date'].str[:4]).agg(
    total_orders=('order_id', 'count'),
    total_quantity=('order_quantity', 'sum'),
    total_sales=('price_sold', 'sum')
).reset_index()

# Plotting a line chart for yearly trends
import plotly.express as px
fig = px.line(
    yearly_trends,
    x='order_date',
    y=['total_orders', 'total_quantity', 'total_sales'],
    title='Yearly Order Trends',
    labels={'order_date.str[:4]': 'Year', 'value': 'Count/Sales'},
)

fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Count/Sales/Orders",
    template='plotly_dark'
)

fig.show()
fig.write_html("Yearly order trends.html")

In [ ]:
# Month over month growth rates
monthly_trends['growth_rate'] = monthly_trends['total_sales'].pct_change() * 100

# Plotting a bar chart for month-over-month growth rates
import plotly.express as px
fig = px.bar(
    monthly_trends,
    x='order_date',
    y='growth_rate',
    title='Month-over-Month Growth Rate',
    labels={'order_date.str[:7]': 'Month', 'growth_rate': 'Growth Rate (%)'},
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Growth Rate (%)",
    template='plotly_dark'
)

fig.show()
fig.write_html("Month over Month Growth Rate.html")

Customer acquisition over time

In [ ]:
# each customer first purchase 
first_orders = df_ord_renamed.groupby('client_id')['order_date'].min().reset_index()
first_orders['acquisition_month'] = first_orders['order_date'].str[:7]
monthly_acquisition = first_orders.groupby('acquisition_month').size().reset_index(name='new_customers_acquired')
monthly_acquisition = monthly_acquisition.sort_values('acquisition_month')
# adding a col to track cumulative customers
monthly_acquisition['cumulative_customers'] = monthly_acquisition['new_customers_acquired'].cumsum()

import plotly.express as px
import plotly.graph_objects as go

# Bar chart for new customers
fig = px.bar(
    monthly_acquisition,
    x='acquisition_month',
    y='new_customers_acquired',
    labels={'acquisition_month': 'Month', 'new_customers_acquired': 'New Customers'},
    title='Monthly New and Cumulative Customer Acquisition'
)

# Add cumulative customer line
fig.add_trace(
    go.Scatter(
        x=monthly_acquisition['acquisition_month'],
        y=monthly_acquisition['cumulative_customers'],
        mode='lines+markers',
        name='Cumulative Customers',
        line=dict(color='orange', width=3)
    )
)

# Improve layout
fig.update_layout(
    xaxis_title='Acquisition Month',
    yaxis_title='Number of Customers',
    legend=dict(x=0.01, y=0.99),
    xaxis_tickangle=-45,
    template='plotly_dark'
)

fig.show()

# saving the interactive graph
fig.write_html("Montly new and cumulative customer acquisition.html")

In [ ]:
first_orders['order_year'] = first_orders['order_date'].str[:4]
first_orders['order_month'] = first_orders['order_date'].str[:7]  # 'YYYY-MM'

# Filter only 2018 and 2019
filtered = first_orders[first_orders['order_year'].isin(['2018', '2019'])]

# Group by year and month to count customers per month
monthly_counts = filtered.groupby(['order_year', 'order_month']).size().reset_index(name='customers_acquired')

# Compute average number of customers acquired per month in each year
avg_2018 = monthly_counts[monthly_counts['order_year'] == '2018']['customers_acquired'].mean()
avg_2019 = monthly_counts[monthly_counts['order_year'] == '2019']['customers_acquired'].mean()

print("Average number of customers acquired:")
print(f"2018: {avg_2018:.2f}")
print(f"2019: {avg_2019:.2f}")


In [ ]:
# Customer acqusition countrywise 2018 vs 2019
first_orders['order_year'] = first_orders['order_date'].str[:4]
filtered = first_orders[first_orders['order_year'].isin(['2018', '2019'])]
merged = filtered.merge(df_cust, on='client_id', how='left')  
countrywise_acquisition = merged.groupby(['order_year', 'country']).size().reset_index(name='customers_acquired')
pivot_table = countrywise_acquisition.pivot(index='country', columns='order_year', values='customers_acquired').fillna(0).astype(int)

import plotly.express as px

melted_df = pivot_table.reset_index().melt(
    id_vars='country',
    value_vars=['2018', '2019'],
    var_name='year',
    value_name='customers_acquired'
)

fig = px.bar(
    melted_df,
    x='country',
    y='customers_acquired',
    color='year',
    barmode='group',
    title='Customer Acquisition Countrywise (2018 vs 2019)',
    labels={'customers_acquired': 'Customers Acquired'}
)

fig.update_layout(
    xaxis_tickangle=-45,
    template='plotly_dark'
)

fig.show()

fig.write_html("Customer Acquisition Countrywise.html")

In [ ]:
first_orders = df_ord_renamed.groupby('client_id', as_index=False)['order_date'].min()
first_orders.rename(columns={'order_date': 'first_order_date'}, inplace=True)
first_orders_info = pd.merge(first_orders, df_cust, on='client_id', how='inner')
affected_countries = ['USA', 'Australia', 'Spain', 'France']
first_orders_info = first_orders_info[first_orders_info['country'].isin(affected_countries)]
first_orders_info['acquisition_month'] = pd.to_datetime(first_orders_info['first_order_date']).dt.to_period('M').astype(str)

monthly_countrywise_acquisition = first_orders_info.groupby(
    ['acquisition_month', 'country']
).size().reset_index(name='customers_acquired')

monthly_countrywise_acquisition = monthly_countrywise_acquisition.sort_values(
    by=['acquisition_month', 'country']
)

print(monthly_countrywise_acquisition)

In [ ]:
orders_with_country = df_ord_renamed.merge(df_cust, on='client_id')

orders_with_country = orders_with_country[
    orders_with_country['country'].isin(['USA', 'Australia', 'Spain', 'France'])
]

orders_with_country['sales_amount'] = (
    orders_with_country['price_sold'] * orders_with_country['order_quantity']
)
orders_with_country['order_year'] = pd.to_datetime(
    orders_with_country['order_date']
).dt.year

orders_filtered = orders_with_country[
    orders_with_country['order_year'].isin([2018, 2019])
]

# Group by country and year and aggregate sales
grouped = orders_filtered.groupby(['country', 'order_year'])['sales_amount'].sum().unstack(fill_value=0)
grouped = grouped.rename(columns={2018: 'sales_2018', 2019: 'sales_2019'})
grouped['loss_amount'] = grouped['sales_2018'] - grouped['sales_2019']

# Sort by loss_amount descending
result = grouped.sort_values(by='loss_amount', ascending=False).reset_index()

print(result)


In [ ]:
# Melt the DataFrame to long format
df_melted = result.melt(
    id_vars='country',
    value_vars=['sales_2018', 'sales_2019'],
    var_name='Year', 
    value_name='Sales'
)

df_melted['Year'] = df_melted['Year'].str.replace('sales_', '')

fig = px.bar(
    df_melted, 
    x='country', 
    y='Sales', 
    color='Year', 
    barmode='group',
    title='Sales Comparison by Country: 2018 vs 2019'
)

fig.update_layout(
    xaxis_title='Country', 
    yaxis_title='Sales Amount',
    template='plotly_dark'
)

fig.show()
fig.write_html("Sales comaprison by country.html")

In [ ]:
# Sort for better visual
result_sorted = result.sort_values('loss_amount')
colors = ['red' if x > 0 else 'green' for x in result_sorted['loss_amount']]

fig = go.Figure(go.Bar(
    x=result_sorted['loss_amount'],
    y=result_sorted['country'],
    orientation='h',
    marker_color=colors
))

fig.update_layout(
    title='Loss/Gain in Sales from 2018 to 2019',
    xaxis_title='Sales Difference (2018 - 2019)',
    yaxis_title='Country',
    template='plotly_dark'
)

fig.show()
fig.write_html("Loss,gain in sales from 2018 to 2019.html")